In [8]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

def get_complete_corner_stats():
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)

    all_teams = []
    current_page = 0

    try:
        while True:
            url = f"https://www.fotmob.com/leagues/47/stats/season/23685/teams/corner_taken_team/premier-league-teams?page={current_page}"
            driver.get(url)

            try:
                WebDriverWait(driver, 15).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, 'div[class*="LeagueSeasonStatsTableRowCSS"]'))
                )
            except:
                print(f"No data found on page {current_page}. Exiting.")
                break

            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(1.5)

            teams = driver.find_elements(By.CSS_SELECTOR, 'div[class*="LeagueSeasonStatsTableRowCSS"]')
            if not teams:
                print(f"No teams found on page {current_page}, stopping.")
                break

            for team in teams:
                try:
                    rank = team.find_element(By.CSS_SELECTOR, 'span[class*="Rank"]').text.strip()
                    name = team.find_element(By.CSS_SELECTOR, 'span[class*="TeamOrPlayerName"]').text.strip()
                    corners = team.find_element(By.CSS_SELECTOR, 'span[class*="StatValue"] span').text.strip()
                    all_teams.append({
                        'Rank': rank,
                        'Team': name,
                        'Corners Taken': corners
                    })
                except Exception as e:
                    print(f"Skipping a row due to error: {e}")

            print(f"Page {current_page} processed with {len(teams)} teams.")
            current_page += 1

    finally:
        driver.quit()

    df = pd.DataFrame(all_teams)
    return df[['Rank', 'Team', 'Corners Taken']]

# Run and output
corner_data = get_complete_corner_stats()
print(corner_data)
corner_data.to_csv('complete_corner_stats.csv', index=False)


Page 0 processed with 10 teams.
Page 1 processed with 10 teams.
No data found on page 2. Exiting.
   Rank                     Team Corners Taken
0     1        Tottenham Hotspur           233
1     2                  Arsenal           230
2     3                Liverpool           229
3     4          Manchester City           225
4     5              Aston Villa           213
5     6                  Chelsea           212
6     7          AFC Bournemouth           208
7     8         Newcastle United           192
8     9                   Fulham           191
9    10        Manchester United           180
10   11   Brighton & Hove Albion           169
11   12           Crystal Palace           156
12   13                Brentford           148
13   14                  Everton           140
14   15          West Ham United           138
15   16              Southampton           134
16   17        Nottingham Forest           133
17   18             Ipswich Town           131
18   19  